In [1]:
import pandas as pd
import pandas_ta as ta 
import numpy as np

In [2]:
path = "Data Historical/AAPL_data.csv"
data = pd.read_csv(path)
data.dtypes

Date         object
Adj Close    object
Close        object
High         object
Low          object
Open         object
Volume       object
dtype: object

In [3]:
numeric_cols = ['Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume']
data[numeric_cols] = data[numeric_cols].apply(pd.to_numeric, errors='coerce')


In [4]:
# Step 2: Calculate RSI for 14, 30, and 200 periods
data['RSI_14'] = ta.rsi(data['Close'], length=14)
data['RSI_30'] = ta.rsi(data['Close'], length=30)
data['RSI_200'] = ta.rsi(data['Close'], length=200)

# Step 3: Calculate Momentum for 10 and 30 periods
data['MOM_10'] = ta.mom(data['Close'], length=10)
data['MOM_30'] = ta.mom(data['Close'], length=30)

# Step 4: Calculate MACD
macd = ta.macd(data['Close'], fast=12, slow=26, signal=9)
data = pd.concat([data, macd], axis=1)  # Add MACD columns to the dataframe

# Step 5: Calculate PROC (Price Rate of Change) for 9 periods
data['PROC_9'] = ta.roc(data['Close'], length=9)

# Step 6: Calculate EMA for 10, 30 periods
data['EMA_10'] = ta.ema(data['Close'], length=10)
data['EMA_30'] = ta.ema(data['Close'], length=30)

data['Pct_Change'] = data['Close'].pct_change()
data['Log_Returns'] = np.log(data['Close'] / data['Close'].shift(1))

data['Volatility_10'] = data['Close'].rolling(window=10).std()
data['Volatility_30'] = data['Close'].rolling(window=30).std()


# Step 7: Display the dataframe with new features
print(data[['Open', 'High', 'Low', 'Close', 'Adj Close', 
            'RSI_14', 'RSI_30', 'RSI_200', 
            'MOM_10', 'MOM_30', 
            'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 
            'PROC_9', 
            'EMA_10', 'EMA_30']].tail())

             Open        High         Low       Close   Adj Close     RSI_14  \
11116  232.119995  232.289993  228.479996  229.979996  229.979996  35.396657   
11117  224.000000  224.419998  219.380005  222.639999  222.639999  29.692784   
11118  219.789993  224.119995  219.789993  223.830002  223.830002  31.616735   
11119  224.740005  227.029999  222.300003  223.660004  223.660004  31.484189   
11120  224.975006  225.630005  222.350098  222.779999  222.779999  30.765178   

          RSI_30    RSI_200     MOM_10     MOM_30  MACD_12_26_9  \
11116  44.192064  53.056888 -13.870010 -12.669998     -3.098927   
11117  40.156453  52.157537 -20.720001 -20.369995     -4.096375   
11118  41.059180  52.289312 -21.169998 -19.209991     -4.736240   
11119  40.967852  52.268642 -18.550003 -19.179993     -5.197146   
11120  40.485619  52.161370 -19.919998 -23.970001     -5.569227   

       MACDh_12_26_9  MACDs_12_26_9    PROC_9      EMA_10      EMA_30  
11116      -2.592732      -0.506195 -5.49803

In [5]:
data.tail()

,Date,Adj Close,Close,High,Low,Open,Volume,RSI_14,RSI_30,RSI_200,...,MACD_12_26_9,MACDh_12_26_9,MACDs_12_26_9,PROC_9,EMA_10,EMA_30,Pct_Change,Log_Returns,Volatility_10,Volatility_30
11116,2025-01-17,229.979996,229.979996,232.289993,228.479996,232.119995,68488300.0,35.396657,44.192064,53.056888,...,-3.098927,-2.592732,-0.506195,-5.498030,236.853013,241.119444,0.007535,0.007507,5.865160,7.920047
11117,2025-01-21,222.639999,222.639999,224.419998,219.380005,224.000000,98070400.0,29.692784,40.156453,52.157537,...,-4.096375,-2.872144,-1.224231,-9.126531,234.268829,239.927221,-0.031916,-0.032436,7.060312,8.963895
11118,2025-01-22,223.830002,223.830002,224.119995,219.789993,219.789993,64126500.0,31.616735,41.059180,52.289312,...,-4.736240,-2.809607,-1.926633,-7.588458,232.370860,238.888691,0.005345,0.005331,7.008649,9.761657
11119,2025-01-23,223.660004,223.660004,227.029999,222.300003,224.740005,60178200.0,31.484189,40.967852,52.268642,...,-5.197146,-2.616410,-2.580735,-7.845074,230.787068,237.906195,-0.000759,-0.000760,6.811638,10.469169
11120,2025-01-24,222.779999,222.779999,225.630005,222.350098,224.975006,24824447.0,30.765178,40.485619,52.161370,...,-5.569227,-2.390793,-3.178434,-5.940472,229.331237,236.930312,-0.003935,-0.003942,5.985241,11.126474


In [6]:
# Step 3: Handle null values
# Drop rows with nulls in price data
data.dropna(subset=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)

# Forward fill nulls in technical indicators
data.fillna(method='ffill', inplace=True)

# Drop any remaining nulls
data.dropna(inplace=True)

# Step 4: Display the cleaned dataframe
print(data.isnull().sum())  # Check for remaining nulls


Date             0
Adj Close        0
Close            0
High             0
Low              0
Open             0
Volume           0
RSI_14           0
RSI_30           0
RSI_200          0
MOM_10           0
MOM_30           0
MACD_12_26_9     0
MACDh_12_26_9    0
MACDs_12_26_9    0
PROC_9           0
EMA_10           0
EMA_30           0
Pct_Change       0
Log_Returns      0
Volatility_10    0
Volatility_30    0
dtype: int64


/tmp/ipykernel_23643/1071741576.py:6: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)


## Define target value

In [7]:
# Define the horizon (e.g., 5 day ahead)
horizon = 5
threshold = 0.02
# Calculate future price change
data['Future_Close'] = data['Close'].shift(-horizon)  # Shift close price forward
data['Price_Change'] = data['Future_Close'] - data['Close']

# Define buy/sell signals
data['Signal'] = np.where(data['Price_Change'] > threshold, 1, 
                         np.where(data['Price_Change'] < -threshold, 0, 2))
data.dropna(subset=['Signal'], inplace=True)  # Remove rows with insignificant moves

# Drop rows with NaN in the target variable (last `horizon` rows)
data.dropna(subset=['Future_Close'], inplace=True)

In [8]:
data.columns

Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume', 'RSI_14',
       'RSI_30', 'RSI_200', 'MOM_10', 'MOM_30', 'MACD_12_26_9',
       'MACDh_12_26_9', 'MACDs_12_26_9', 'PROC_9', 'EMA_10', 'EMA_30',
       'Pct_Change', 'Log_Returns', 'Volatility_10', 'Volatility_30',
       'Future_Close', 'Price_Change', 'Signal'],
      dtype='object')

In [9]:
data.to_csv('final_data.csv', index = False)
